In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Final Data/"

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_tr.pkl", "rb")
ca_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_tr.pkl", "rb")
en_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_tr.pkl", "rb")
fr_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_tr.pkl", "rb")
it_data_tr = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_tr.pkl", "rb")
de_data_tr = pickle.load(sound_lists_pkl_in)



In [0]:
#Loading test pickle files
sound_lists_pkl_in = open(dirpath+"ca_data_padded_te.pkl", "rb")
ca_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"en_data_padded_te.pkl", "rb")
en_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"fr_data_padded_te.pkl", "rb")
fr_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"it_data_padded_te.pkl", "rb")
it_data_te = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"de_data_padded_te.pkl", "rb")
de_data_te = pickle.load(sound_lists_pkl_in)

In [0]:
final_data_tr = np.concatenate((ca_data_tr, en_data_tr, fr_data_tr, it_data_tr, de_data_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_te, en_data_te, fr_data_te, it_data_te, de_data_te), axis = 0)

In [8]:
print(final_data_tr.shape)
print(final_data_te.shape)

(10000, 2)
(2500, 2)


In [0]:
del(de_data_te)
del(ca_data_te)
del(en_data_te)
del(fr_data_te)
del(it_data_te)

In [0]:
del(de_data_tr)
del(ca_data_tr)
del(en_data_tr)
del(fr_data_tr)
del(it_data_tr)

In [0]:
# final_data_tr

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_tr.pkl", "wb")
# pickle.dump(final_data_tr, data_padded_pkl)

In [0]:
# data_padded_pkl = open(dirpath+"final_data_te.pkl", "wb")
# pickle.dump(final_data_te, data_padded_pkl)

In [15]:

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)

final_labels_tr = indices

[4 2 0 ... 2 0 4]


In [16]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)

final_labels_te = indices

[3 2 1 ... 1 3 3]


In [17]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape




(972, 40)

In [18]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(972, 40)

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten
from keras.layers import Dense, Permute, Reshape, Input
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet
from keras.models import Model
import keras

In [0]:
del(final_data_tr)
del(final_data_te)

In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy = to_categorical(cvy)
ty = to_categorical(ty)

In [0]:
# input_shape = (40,972,1)
input_shape2 = (972,40,1)


In [25]:
#Resnet Model
base_model = MobileNet(weights = None, include_top= False, input_shape=input_shape2, alpha=1.0, dropout = 0.9)

# for layer in base_model.layers:
#   layer.trainable = False
 
x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Dropout(0.9)(x)
  
# model.add(Permute((2,1,3)))

# bs, x, y, c = model.layers[-1].output_shape

# model.add(Reshape([x,y*c]))

# model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# x = Permute((2, 1, 3))(x)

_x, _y, _c = [int(s) for s in x._shape[1:]]
x = Reshape((_x, _y*_c))(x)

# x = LSTM(256, return_sequences=True)(x)

x = LSTM(512, activation="tanh", return_sequences=False, dropout=0.7)(x)

# x = Flatten()(x)

# x = Dense(1024, activation="relu")(x)

predictions = Dense(5, activation="softmax")(x)

model = Model(inputs = base_model.input, output = predictions)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 972, 40, 1)        0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 973, 41, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 486, 20, 32)       288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 486, 20, 32)       128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 486, 20, 32)       0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 486, 20, 32)       288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 486, 20, 32)       128       
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0002),
             metrics=["accuracy"])

In [29]:
model.fit(tx.reshape(-1,972,40,1), ty,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(cvx.reshape(-1,972,40,1),cvy)
         )
          

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 48s 6ms/step - loss: 1.0553 - acc: 0.5490 - val_loss: 2.2458 - val_acc: 0.3445
Epoch 2/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.9644 - acc: 0.5925 - val_loss: 2.1013 - val_acc: 0.3900
Epoch 3/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.9006 - acc: 0.6210 - val_loss: 1.6868 - val_acc: 0.4265
Epoch 4/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.8580 - acc: 0.6413 - val_loss: 1.5789 - val_acc: 0.4690
Epoch 5/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.8182 - acc: 0.6535 - val_loss: 1.3090 - val_acc: 0.5200
Epoch 6/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.7603 - acc: 0.6771 - val_loss: 1.4716 - val_acc: 0.5185
Epoch 7/100
8000/8000 [==============================] - 47s 6ms/step - loss: 0.6970 - acc: 0.7115 - val_loss: 1.2842 - val_acc: 0.5605


In [0]:
y_pred = model.predict(tx.reshape(-1,972,40,1))
matrix = confusion_matrix(ty.argmax(axis=1), y_pred.argmax(axis=1))

In [0]:
matrix

In [0]:
y_pred

In [0]:
tx[0].shape